# TP - ES Problèmes inverses

Vincent Duval, Nicolas Desassis

## Exercice 3 - le problème des pixels manquants (inpainting)
On reprend l'exercice vu en TD pour expérimenter.

using Images, Plots, TestImages,Printf
using StatsBase

In [ ]:
img=Float32.(Gray.(testimage("coffee"))) # house,coffee, chelsea
plot(Gray.(img),title="Image originale")

In [ ]:
nb=length(img) # Nombre de pixels
tauxpix=10 # pourcentage de pixels morts  (A MODIFIER)
nd=Int(round(nb*tauxpix/100)) # Nombre de pixels morts

#Tire une sous-selection aléatoire
I=1:nb
J=sample(I,nd;replace=false) # Indices des pixels morts
K=[j for j in 1:nb if j ∉ J ] # Indices des pixels vivants

y=img[K] # Observation
display(@sprintf("Nombre total de pixels: %d",nb))
display(@sprintf("Nombre de pixels manquants: %d (%.2f %%)",nd,tauxpix))

## Solution de norme $\ell^2$ minimale

On résout $$u_0 =\mathrm{argmin}_u \|u\|_{\ell^2} \quad \mbox{s.c.}\quad Au=y$$

In [ ]:
# Solution de norme minimale
u0=copy(img)
u0[J].=0.
plot(Gray.(u0))

In [ ]:
function proj_affine!(u,y,K)
    # Projette orthogonalement sur l'espace Au=y
    # Attention: cette fonction modifie son premier argument
    u[K].=y
end

## Régularisation par norme de Sobolev

On considère à la place une autre régularisation pour choisir la solution
$$ u_1=\mathrm{argmin}_u \frac{1}{2}\|Du\|_{\ell^2}^2 \quad \mbox{s.c.}\quad Au=y$$
où $D$ est un opérateur de gradient (avec conditions de bord périodiques, pour simplifier):
$$\forall j \in \{1,\ldots,N\},\quad(Du)_{i,j,1}=\begin{cases} u_{i+1,j}-u_{i,j} &\mbox{pour $1\leq i \leq M-1$}, \\
                u_{1,j}-u_{M,j} &\mbox{pour $i=M$}.\end{cases}
                $$
$$\forall i \in \{1,\ldots,M\},\quad (Du)_{i,j,2}=\begin{cases} u_{i,j+1}-u_{i,j} &\mbox{pour  $1\leq j \leq N-1$, } \\
                u_{i,1}-u_{i,N} &\mbox{pour $j=N$.}\end{cases}
                $$


On veut résoudre ce problème de variation par la méthode de gradient projeté:

$$ u_{k+1} = P_C\left(u_k-\tau D^*Du_k\right)$$

où l'ensemble des points admissibles est $C=\{u \in\mathbb{R}^{MN} | Au =y \}$.

In [ ]:
# Quelques fonctions auxiliaires (évaluer la cellule pour les charger)

function applyD!(Du,u)
    """
    Opérateur de gradient avec conditions de bord périodiques
    Attention: cette fonction modifie son premier argument
    """
    (M,N)=size(u)
    fill!(Du,zero(eltype(Du))) # remise à zéro de Du
    #Premier argument
   for j in  1:N
        for i in 1:M-1
           Du[i,j,1]=u[i+1,j]-u[i,j] 
        end 
        Du[M,j,1]=u[1,j]-u[M,j]
    end
    
    # Deuxième argument 
    for j in 1:N-1
        for i in 1:M
            Du[i,j,2]=u[i,j+1]-u[i,j]
        end
    end
    for i in 1:M
       Du[i,N,2]=u[i,1]-u[i,N] 
    end
end

function applyDstar!(Dsv,v)
    """
    Calcul de l'adjoint de D.
    Attention: cette fonction modifie son premier argument
    """
    # Typiquement v=Du
    (M,N,P)=size(v)
    fill!(Dsv,zero(eltype(Dsv))) # remise à zéro de Dsv
    
    for j in  1:N
        for i in 2:M
                Dsv[i,j] += v[i-1,j,1]-v[i,j,1] # l'opérateur += ajoute le membre de droite à celui de gauche
        end
        Dsv[1,j] +=v[M,j,1]-v[1,j,1]
    end
    
    for j in 2:N
        for i in 1:M
            Dsv[i,j] += v[i,j-1,2]-v[i,j,2]
        end
    end
    for i in 1:M
       Dsv[i,1] += v[i,N,2]-v[i,1,2] 
    end
    
end

# test de l'adjoint (A faire une fois pour toute mais C'EST IMPORTANT pour valider que le calcul de l'adjoint est correct)
# On vérifie la formule de l'adjoint sur (attention aux overflows: on prend de petits entiers)
(n1,n2)=5,7
t=rand(1:255,n1,n2)
Dt=zeros(Int,n1,n2,2)
applyD!(Dt,t)
v=rand(1:255,n1,n2,2)
Dsv=zeros(size(t))
applyDstar!(Dsv,v)

abs(sum(Dsv.*t)-sum(v.*Dt))

In [ ]:
#Descente de gradient projeté (COMPLETER L'ALGORITHME CI-DESSOUS)
tau=0.01
nbiter=800
u=zeros(size(img))
Du=zeros(size(u)...,2)
DsDu=zeros(size(u))
nrj=zeros(nbiter)
for k in 1:nbiter
    applyD!(Du,u) # Du
    applyDstar!(DsDu,Du) # D*(Du)
    # u.=   # descente de gradient A COMPLETER
    proj_affine!(u,y,K) # projection sur la contrainte
    
    #Calcul de l'énergie
    applyD!(Du,u) # Du
    nrj[k]=sum(Du.^2)/2
end
plot(1:nbiter,nrj,xlabel="Itérations",ylabel="Energie")

In [ ]:
p0=plot(Gray.(u0),xlabel="Norme minimale")
p1=plot(Gray.(u),xlabel="Image reconstruite")
p2=plot(Gray.(img),xlabel="Image originale")
p3=plot(Gray.(u),xlabel="Image reconstruite")
plot(p0,p1,p2,p3,size=(1000,1000))